# AP1roG calculations

In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

## Setting up the molecular Hamiltonian in the canonical RHF spinor basis

In [2]:
molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/ch4_crawdad.xyz", 0)  # create a neutral molecule
N = molecule.numberOfElectrons()
N_P = N // 2

In [3]:
spinor_basis = gqcpy.RSpinOrbitalBasis(molecule, "STO-3G")
K = spinor_basis.numberOfSpatialOrbitals()

S = spinor_basis.quantizeOverlapOperator().parameters()

In [4]:
sq_hamiltonian = gqcpy.SQHamiltonian.Molecular(spinor_basis, molecule)  # 'sq' for 'second-quantized'

The current Hamiltionian is expressed in the non-orthogonal AO basis. This is exactly what we need to start an RHF calculation.

In [5]:
environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, sq_hamiltonian, S)
solver = gqcpy.RHFSCFSolver.DIIS()

objective = gqcpy.DiagonalRHFFockMatrixObjective(sq_hamiltonian)  # use the default threshold of 1.0e-08

Using this objective makes sure that the optimized expansion coefficients yield a diagonal Fock matrix, so they will represent the canonical RHF spinor basis.

In [6]:
rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()
C = rhf_parameters.coefficientMatrix()

Since we have the canonical RHF spinor expansion coefficients now, we can transform the underlying spinor basis and then re-quantize the molecular Hamiltonian, in order to let both instances be in-sync with their basis transformations.

However, gqcpy offers a different approach, transforming the spinor basis and Hamiltonian in one call.

In [7]:
gqcpy.basisTransform(spinor_basis, sq_hamiltonian, C)

Right now, the spinor basis and Hamiltonian are expressed in the canonical RHF spinors.

## AP1roG calculations

AP1roG has also been formulated as a QCModel/QCMethod. Therefore, we should create a solver (which is able to solve the AP1roG PSEs), an associated environment and an objective.

For AP1roG, the solver is a non-linear equation solver.

In [8]:
environment = gqcpy.PSEnvironment.AP1roG(sq_hamiltonian, N_P)
solver = gqcpy.NonLinearEquationSolver.Newton()

In [9]:
qc_structure = gqcpy.AP1roG(sq_hamiltonian, N_P).optimize(solver, environment)

In [10]:
print(qc_structure.groundStateEnergy())

-53.249920240098675


In [11]:
print(qc_structure.groundStateParameters().geminalCoefficients().asMatrix())

[[ 1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00 -7.790e-04 -7.792e-04 -7.788e-04 -1.948e-03]
 [ 0.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00 -1.415e-02 -1.420e-02 -1.413e-02 -2.450e-02]
 [ 0.000e+00  0.000e+00  1.000e+00  0.000e+00  0.000e+00 -4.887e-02 -2.696e-02 -1.432e-02 -1.867e-02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00  0.000e+00 -1.288e-02 -2.266e-02 -5.524e-02 -1.863e-02]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.000e+00 -2.859e-02 -3.993e-02 -2.111e-02 -1.872e-02]]


## vAP1roG calculations

We have also extended the AP1roG method to be variationally optimized, resulting in the vAP1roG method. In short, what this method does is analogous to AP1roG, but after determining the optimal geminal coefficients, a set of optimal Lagrange multipliers is also searched for.

Since these Lagrange multipliers are determined through solving a linear equation, we will have to supply a linear equations solver to the vAP1roG method.

In [12]:
non_linear_environment = gqcpy.PSEnvironment.AP1roG(sq_hamiltonian, N_P)
non_linear_solver = gqcpy.NonLinearEquationSolver.Newton()

linear_solver = gqcpy.LinearEquationSolver.ColPivHouseholderQR()

In [13]:
qc_structure = gqcpy.vAP1roG(sq_hamiltonian, N_P).optimize(non_linear_solver, non_linear_environment, linear_solver)

In [14]:
print(qc_structure.groundStateEnergy())

-53.249920240098675


In [15]:
print(qc_structure.groundStateParameters().geminalCoefficients().asMatrix())

[[ 1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00 -7.790e-04 -7.792e-04 -7.788e-04 -1.948e-03]
 [ 0.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00 -1.415e-02 -1.420e-02 -1.413e-02 -2.450e-02]
 [ 0.000e+00  0.000e+00  1.000e+00  0.000e+00  0.000e+00 -4.887e-02 -2.696e-02 -1.432e-02 -1.867e-02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00  0.000e+00 -1.288e-02 -2.266e-02 -5.524e-02 -1.863e-02]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.000e+00 -2.859e-02 -3.993e-02 -2.111e-02 -1.872e-02]]


In [16]:
print(qc_structure.groundStateParameters().lagrangeMultipliers())

[[-0.001 -0.001 -0.001 -0.002]
 [-0.014 -0.014 -0.014 -0.025]
 [-0.049 -0.027 -0.014 -0.019]
 [-0.013 -0.023 -0.055 -0.019]
 [-0.029 -0.04  -0.021 -0.019]]


## Orbital optimization for vAP1roG

We've also implemented an second-order orbital optimizer that uses a Newton step in every iteration. (The API isn't quite up-to-par yet.)

The orbital optimize requires an initial guess.

In [17]:
G_initial = qc_structure.groundStateParameters().geminalCoefficients()

optimizer = gqcpy.AP1roGLagrangianNewtonOrbitalOptimizer(G_initial, oo_convergence_threshold=1.0e-04)
optimizer.optimize(spinor_basis, sq_hamiltonian)

We can see that the electronic energy has lowered due to the orbital optimization.

In [18]:
print(optimizer.get_electronic_energy())

-53.285570717469284


The converged geminal coefficients and multipliers can be found, too.

In [19]:
print(optimizer.get_geminal_coefficients().asMatrix())

[[ 1.     0.     0.     0.     0.    -0.001 -0.001 -0.001 -0.001]
 [ 0.     1.     0.     0.     0.    -0.007 -0.007 -0.007 -0.091]
 [ 0.     0.     1.     0.     0.    -0.091 -0.007 -0.007 -0.007]
 [ 0.     0.     0.     1.     0.    -0.007 -0.007 -0.091 -0.007]
 [ 0.     0.     0.     0.     1.    -0.007 -0.091 -0.007 -0.007]]


In [20]:
print(optimizer.get_multipliers())

[[-0.001 -0.001 -0.001 -0.001]
 [-0.007 -0.007 -0.007 -0.091]
 [-0.091 -0.007 -0.007 -0.007]
 [-0.007 -0.007 -0.091 -0.007]
 [-0.007 -0.091 -0.007 -0.007]]
